In [1]:
import pandas as pd
import os, json
from unidecode import unidecode

import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;

In [2]:
def getBeautifyText(text):
     return unidecode(text.replace('\n', ' '))

In [3]:
# Get only question
# Get all json file from data directory
path_to_json = 'data/'
columns = ['question']
questionAndAnswerDf = pd.DataFrame(columns=columns)
indexCount = 0
for pos_json in os.listdir(path_to_json):
    if pos_json.endswith('.json'):
        with open(path_to_json + pos_json, encoding="utf8") as json_file:
            data = json.load(json_file)
            for index, questionObject in enumerate(data):
                questionAndAnswerDf.loc[indexCount] = [getBeautifyText(questionObject["question"])]
                indexCount += 1

In [4]:
questionAndAnswerDf.head()

,question
0,How do you compare Methane Ppm and carbon(iv) ...
1,Equatorial Bulge and its implications on the c...
2,Where oil is used for [closed]
3,Rain Water vs Sprinkler Irrigation
4,Climate Change in the North


In [5]:
data_text = questionAndAnswerDf
for idx in range(len(data_text)):
    
    #go through each word in each data_text row, remove stopwords, and set them on the index.
    data_text.iloc[idx]['question'] = [word for word in data_text.iloc[idx]['question'].split(' ') if word not in stopwords.words()];
    
    #print logs to monitor output
    if idx % 1000 == 0:
        sys.stdout.write('\rc = ' + str(idx) + ' / ' + str(len(data_text)));
#save data because it takes very long to remove stop words
pickle.dump(data_text, open('data_text.dat', 'wb'))
#get the words as an array for lda input
train_questions = [value[0] for value in data_text.iloc[0:].values];

c = 3000 / 3233

In [6]:
num_topics = 10;

In [7]:
id2word = gensim.corpora.Dictionary(train_questions);
corpus = [id2word.doc2bow(text) for text in train_questions];
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics);

In [8]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [9]:
get_lda_topics(lda, num_topics)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,climate,climate,CO2,New,What,The,The,global,The,It's
1,The,change,climate,global,How,Climate,A,How,climate,Climate
2,change,How,The,Climate,CO2,warming,How,climate,change,Global
3,What,ice,change,Why,better,Why,climate,change,Why,Warming
4,How,CO2,My,A,climate,Global,Climate,temperature,greenhouse,CO2
5,global,global,Let's,CO2,IPCC,climate,world,good,global,Change
6,Is,warming,temperature,Global,increasing,How,Why,The,warming,I
7,Why,-,global,us,effect,CO2,change,Is,future,The
8,warming,ways,How,Change,The,Change,data,warming,Climate,Why
9,Earth,Sea,cities,The,change,Earth,CO2,warming?,Are,What


In [10]:
#the count vectorizer module needs string inputs, not array, so I join them with a space. This is a very quick operation.
train_questions_sentences = [' '.join(text) for text in train_questions]

In [11]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000);
x_counts = vectorizer.fit_transform(train_questions_sentences);

In [12]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [13]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [14]:
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd');
#fit the model
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=10, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [15]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [16]:
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,climate,warming,co2,the,it,ice,what,why,heat,greenhouse
1,change,global,increase,sun,ocean,ages,changed,temperature,waste,effect
2,survey,cooling,increasing,science,cooling,melting,earth,earth,work,gases
3,how,oceans,made,case,us,age,environment,rising,trolls,gas
4,denial,cycles,man,world,sun,arctic,weather,recycle,holding,earth
5,is,how,cause,debate,methane,loss,wrong,how,how,gasses
6,science,cause,emissions,truth,albedo,greenland,mean,this,atmosphere,real
7,north,no,pollutant,emissivity,bad,dry,hell,average,ocean,there
8,closed,is,issue,top,degrees,sea,steve,surface,traps,science
9,questionnaire,answer,begets,end,ozone,gaining,price,ocean,sources,how
